In [1]:
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import numpy as np
import torch

%autosave 60
%load_ext line_profiler

Autosaving every 60 seconds


In [2]:
from src.utils.pipeline import Pipeline
from torch.utils.data import Dataset, DataLoader
from src.utils.datasets import OrientatedGaussians

from src.supervised_gcal.models import get_lissom 
from src.utils.images import plot_matrix, debug, plot_array_matrix

In [ ]:
size = 24
count = 10000
epochs = 1
orientations = 25

In [4]:
lissom, optimizer, _ = get_lissom(retinal_density=size, lgn_params='lgn', rlissom_params='rlissom', optim_params='optim')
display(lissom)

In [6]:
gaussians_inputs = OrientatedGaussians(size=size, length=count)
train_loader = DataLoader(gaussians_inputs, shuffle=True, pin_memory=False)
pipe = Pipeline(lissom, optimizer, cuda=True, log_interval=10)

In [ ]:
pipe.train(train_loader, epoch=0)

In [ ]:
# from src.utils.datasets import CKDataset
# size = 96
# ck_dataset = CKDataset()
# from torch.utils.data import DataLoader
# train_loader = DataLoader(ck_dataset, shuffle=True, pin_memory=True)

In [ ]:
from src.utils.pipeline import Pipeline
# lissom.cuda()
for inp, _ in train_loader:
    inp = torch.autograd.Variable(inp)
    inp = Pipeline.process_input(inp)
#     inp = inp.cuda()
    act = lissom(inp)
    plot_matrix(np.reshape(act.data.cpu().numpy(), cortex_shape))
    break

In [ ]:
from src.supervised_gcal.orientation_maps import OrientationMap
from src.supervised_gcal.orientation_maps import get_oriented_lines

keys_arrays = get_oriented_lines(size, orientations=orientations)

In [ ]:
keys_arrays = {k: map(torch.autograd.Variable, lines) for k, lines in keys_arrays.items()}

In [ ]:
om = OrientationMap(model=lissom, inputs=keys_arrays)

orientation_map = om.get_orientation_map()
orientation_hist = om.get_orientation_hist()

In [ ]:
from src.supervised_gcal.orientation_maps import plot_orientation_map
plot_orientation_map(orientation_map)
plt.colorbar()
plt.show()

In [ ]:
from src.supervised_gcal.orientation_maps import plot_orientation_hist, metrics_orientation_hist
plot_orientation_hist(orientation_hist)
plt.show()

mean, std = metrics_orientation_hist(orientation_hist)
print("Promedio: %{:.2f}".format(mean*100))
print("Standard deviation: %{:.2f}".format(std*100))

In [ ]:
d